##### https://www.geeksforgeeks.org/python-create-video-using-multiple-images-using-opencv/

In [ ]:
import cv2, os, re

image_folder = '/home/iot-class/Capstone/CARLA_tutorial/test_images/'
video_name = 'real_video.mp4'

images = sorted([img for img in os.listdir(image_folder) if "pointcloud" in img.lower()], key=lambda x: int(re.search(r'\d+', x).group()))
print("Images:", images)


Images: ['000067_semseg.png', '000068_semseg.png', '000069_semseg.png', '000070_semseg.png', '000071_semseg.png', '000072_semseg.png', '000073_semseg.png', '000074_semseg.png', '000075_semseg.png', '000076_semseg.png', '000077_semseg.png', '000078_semseg.png', '000079_semseg.png', '000080_semseg.png', '000081_semseg.png', '000082_semseg.png', '000083_semseg.png', '000084_semseg.png', '000085_semseg.png', '000086_semseg.png', '000087_semseg.png', '000088_semseg.png', '000089_semseg.png', '000090_semseg.png', '000091_semseg.png', '000092_semseg.png', '000093_semseg.png', '000094_semseg.png', '000095_semseg.png', '000096_semseg.png', '000097_semseg.png', '000098_semseg.png', '000099_semseg.png', '000100_semseg.png', '000101_semseg.png', '000102_semseg.png', '000103_semseg.png', '000104_semseg.png', '000105_semseg.png', '000106_semseg.png', '000107_semseg.png', '000108_semseg.png', '000109_semseg.png', '000110_semseg.png', '000111_semseg.png', '000112_semseg.png', '000113_semseg.png', '000

In [33]:
# Set frame from the first image
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# Video writer to create .mp4 file
fps = 1 / 0.05
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Appending images to video
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

# Release the video file
video.release()
cv2.destroyAllWindows()
print("Video generated successfully!")

Video generated successfully!


##### https://github.com/ageitgey/face_recognition/blob/master/examples/facerec_from_video_file.py

### Currently, facial recognition does not work well as the resolution of the videos and faces are not sufficient for this model.
TODO: use newer YOLO model or another pipeline for better results.

In [ ]:
import face_recognition
import cv2
from collections import Counter


# Open the input movie file
input_movie = cv2.VideoCapture("real_video3.mp4")
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

# Create an output movie file (make sure resolution/frame rate matches input video!)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_movie = cv2.VideoWriter('fr3test.mp4', fourcc, 20, (1600, 1200))

# Load some sample pictures and learn how to recognize them.
four_image = face_recognition.load_image_file("people/0004.png")
four_encoding = face_recognition.face_encodings(four_image)[0]

nineteen_image = face_recognition.load_image_file("people/0019.png")
nineteen_encoding = face_recognition.face_encodings(nineteen_image)[0]

twentyone_image = face_recognition.load_image_file("people/0021.png")
twentyone_encoding = face_recognition.face_encodings(twentyone_image)[0]

known_faces = [
    four_encoding,
    nineteen_encoding,
    twentyone_encoding
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
frame_number = 0
face_count = Counter()

while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # results = model(frame)

    # for result in results:
    #     for box in result.boxes:

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame, model='cnn')
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)

        # If you had more than 2 faces, you could make this logic a lot prettier
        # but I kept it simple for the demo
        name = None
        if match[0]:
            name = "four"
        elif match[1]:
            name = "nineteen"
        elif match[2]:
            name = "twentyone"

        face_names.append(name)
        face_count[name] += 1

    # Label the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if not name:
            continue

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    print("Writing frame {} / {}; {}".format(frame_number, length, face_count))
    output_movie.write(frame)

# All done!
input_movie.release()
output_movie.release()
cv2.destroyAllWindows()

In [6]:
print(face_count)

Counter({'four': 25, None: 2})
